之所以起这个名字，实际上股票买卖问题的本质即，对于数组中的所有元素，想找到两个元素，满足
- num[i] - num[j] 这一 metric 最大，即 max(num[i] - num[j])；
- i 必须在 j 之后，即 i > j;

这是一个标准的最优化目标，一个目标函数，一个 constraint；不论这两个事情怎么换（比如，改成 max(num[i] * num[j])，或者改成 i > j），其思路是不变的。

121. 买卖股票的最佳时机

https://leetcode.cn/problems/best-time-to-buy-and-sell-stock/description/?envType=study-plan-v2&envId=top-interview-150

In [ ]:
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        n, best = len(prices), 0
        if n < 2:
            return best
        for i in range(n):
            for j in range(i + 1, n):
                diff = prices[j] - prices[i]
                best = max(diff, best)
        return best

如上自然是最好想的写法，非常直觉明了，但是时间复杂度是二阶的。那么更好的做法是什么？这里要反思的是，哪些操作是冗余的，很明显，我们对于元素的访问是冗余的，也就是说访问的次数**本可以更少的**。

那么，什么算法可以减少访问次数呢？增加指针？使用哈希？我们现在要做的变换是，将 max(num[i] - num[j]) 这一目标改为 max(num[i]) 和 min(num[j])，且 i > j。实际上，就是维护“到目前为止的最低买入价”，并实时更新最大利润即可。

In [ ]:
from typing import List

class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        min_price = float("inf")
        max_profit = 0

        for p in prices:
            if p < min_price:
                min_price = p
            else:
                max_profit = max(max_profit, p - min_price)

        return max_profit

或者如下，在实现上只有一次更新的相位差。但要学会正无穷的写法，即 float("inf")。

In [ ]:
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        best, lowest = 0, 100000
        if len(prices) < 2:
            return best
        for price in prices:
            best = max(price - lowest, best)
            lowest = min(lowest, price)
        return best